In [4]:
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
print(help(tradeapi))

Help on package alpaca_trade_api:

NAME
    alpaca_trade_api

PACKAGE CONTENTS
    __main__
    common
    entity
    polygon (package)
    rest
    stream
    stream2

VERSION
    0.26

FILE
    /home/snoelover18/anaconda3/lib/python3.5/site-packages/alpaca_trade_api/__init__.py


None


# Trading bot using Naive_bayes for decision making
We will build a trading bot using alpaca api and a single machine learning method to make trading decisions. 
This will be a long term trading bot not a HFT or day trading bot.

Alpaca Dashboard: 
https://app.alpaca.markets/paper/dashboard/overview

# Agent
Sources:

https://github.com/alpacahq/alpaca-trade-api-python 

https://docs.alpaca.markets/api-documentation/

https://hackernoon.com/easily-build-a-stock-trading-bot-using-broker-api-6bfbdc8d48c9

To Do List:
- [x] Connect to api
- [X] Make a trade
- [X] Make a bot to do a few trades during the day
- [ ] Check that bot is working properly after a few days


In [2]:
class Agent():
    def __init__(self):
        base_url = 'https://paper-api.alpaca.markets'
        api_key_id = 'PKQ2P1EVP13ETNHEGER3'
        api_secret = '8UwnWrkcj0bp9duzoW4XUljvTOxxlLbWw7f2CBYJ'

        self.api = tradeapi.REST(
            base_url=base_url,
            key_id=api_key_id,
            secret_key=api_secret
        )
        self.account = self.api.get_account()
        self.assets = self.api.list_assets()
        print("Status: ", self.account.status)
        print("Cash: $", self.account.cash)
        print('Clock: ', self.api.get_clock())

    
    def status(self):
        '''
        Return:
            - Displays status of agent
        '''
        print('Market Status: ', self.api.get_clock())
        print("Status: ", self.account.status)
        print("Positions: ", self.api.list_positions())
        
    def get_stock_status(self, symbol):
        for exchange in self.assets:
            if exchange.symbol == symbol:
                print(exchange)
                
    def get_orders(self):
        '''
        Return: 
            - List of currently placed orders
        '''
        return self.api.list_orders()
    
    def get_positions(self):
        '''
        Return: 
            - List of currently owned positions
        '''
        return self.api.list_positions()
        
        
    def get_stock_history(self, symbol, time):
        '''
        Parameters:
            symbol - stock symbol for lookupo
            time - time symbol (e.g minute, 1Min, 5Min, 15Min, day or 1D. minute)
        Return:
            - displays plots of Open vs Close, High vs Low      
        '''
        tsla_data = self.api.get_barset(symbol , time, limit = 100).df['TSLA']
        tsla_data['open'].plot()
        tsla_data['close'].plot()
        plt.title('open vs close')
        plt.show()
        tsla_data['high'].plot()
        tsla_data['low'].plot()
        plt.title('High vs Low')
        plt.show()   
        
    def order(self, symbol, quantity, time_in_force):
        '''
        Parameters:
            symbol - stock symbol for lookupo
            quantity - how many stocks you are buying
            time_in_force - Order type
                1. dat: The order is good for the day, and it will be canceled automatically at the end of Regular Trading Hours if unfilled.
                2. gtc: The order is good until canceled.
                3. opg: The order is placed at the time the market opens. The order will be accepted if it is received before 9:15AM (ET). 
                        The order can be cancelled after 9:15AM, but it cannot be edited. After 9:28AM, OPG orders cannot be edited or cancelled.
                        Any unfilled orders after opening of the market will be cancelled. 
                        If you submit OPG orders during the market hours, it will appear as “rejected” in your dashboard.
        Return:
            order - contents and status of the order placed witht the call
        '''
        order = self.api.submit_order(symbol, quantity,'buy','market',time_in_force)
        return order
    
    def sell(self, symbol, quantity, time_in_force):
        '''
        Parameters:
            symbol - stock symbol for lookupo
            quantity - how many stocks you are buying
            time_in_force - Order type
                1. dat
                2. gtc
                3. opg
        Return:
            order - contents and status of the order placed witht the call
        '''
        try:
            openPosition = self.api.get_position(symbol)
            order = self.api.submit_order(symbol, openPosition,'sell','market',time_in_force)
            return order
        except:
            print("No position to sell")
            return null

agent = Agent()

Status:  ACTIVE
Cash: $ 99432.06
Clock:  Clock({   'is_open': False,
    'next_close': '2019-03-12T16:00:00-04:00',
    'next_open': '2019-03-12T09:30:00-04:00',
    'timestamp': '2019-03-12T00:45:30.371043468-04:00'})


In [24]:
agent.status()
agent.get_stock_status('TSLA')


Market Status:  Clock({   'is_open': False,
    'next_close': '2019-03-12T16:00:00-04:00',
    'next_open': '2019-03-12T09:30:00-04:00',
    'timestamp': '2019-03-12T03:27:14.366706536-04:00'})
Status:  ACTIVE
Positions:  [Position({   'asset_class': 'us_equity',
    'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
    'avg_entry_price': '283.97',
    'change_today': '0.0215394361735825',
    'cost_basis': '567.94',
    'current_price': '290.25',
    'exchange': 'NASDAQ',
    'lastday_price': '284.13',
    'market_value': '580.5',
    'qty': '2',
    'side': 'long',
    'symbol': 'TSLA',
    'unrealized_intraday_pl': '12.56',
    'unrealized_intraday_plpc': '0.0221150121491707',
    'unrealized_pl': '12.56',
    'unrealized_plpc': '0.0221150121491707'})]
Asset({   'asset_class': 'us_equity',
    'exchange': 'NASDAQ',
    'id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
    'status': 'active',
    'symbol': 'TSLA',
    'tradable': True})


In [8]:
checked = []
for order in agent.get_orders():
    if order.symbol not in checked:
        agent.get_stock_history(order.symbol , 'minute')
    checked.append(order.symbol)

In [5]:
        base_url = 'https://paper-api.alpaca.markets'
        api_key_id = 'PKQ2P1EVP13ETNHEGER3'
        api_secret = '8UwnWrkcj0bp9duzoW4XUljvTOxxlLbWw7f2CBYJ'

        api = tradeapi.REST(
            base_url=base_url,
            key_id=api_key_id,
            secret_key=api_secret
        )

In [6]:
api.polygon.all_tickers()

[Ticker({   'day': {   'c': 99.79,
                'h': 100.385,
                'l': 98.65,
                'o': 99.02,
                'v': 1327670,
                'vw': 99.7258},
     'lastTrade': {   'c': [],
                      'i': 587255786,
                      'p': 99.84,
                      's': 100,
                      't': 1552502772605006848,
                      'x': 11},
     'min': {   'c': 99.78,
                'h': 99.945,
                'l': 99.78,
                'o': 99.945,
                'v': 5888,
                'vw': 99.8709},
     'prevDay': {   'c': 98.11,
                    'h': 99.27,
                    'l': 98.07,
                    'o': 98.83,
                    'v': 2054484,
                    'vw': 0},
     'ticker': 'WP',
     'todaysChange': 1.73,
     'todaysChangePerc': 1.763,
     'updated': 1552502772605006848}), Ticker({   'day': {   'c': 0.9701,
                'h': 0.9999,
                'l': 0.94,
                'o': 0.98,


In [36]:
agent.order('TSLA', 100, 'gtc')


Order({   'asset_class': 'us_equity',
    'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
    'canceled_at': None,
    'client_order_id': '008e7320-5b09-4651-8e1f-b05ef67bb312',
    'created_at': '2019-03-12T07:59:15.542790654Z',
    'expired_at': None,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '31f97922-65b3-4cca-a4db-888054c79d45',
    'limit_price': None,
    'order_type': 'market',
    'qty': '100',
    'side': 'buy',
    'status': 'new',
    'stop_price': None,
    'submitted_at': '2019-03-12T07:59:15.536242867Z',
    'symbol': 'TSLA',
    'time_in_force': 'gtc',
    'type': 'market',
    'updated_at': '2019-03-12T07:59:15.557951461Z'})

In [56]:
x = [3, 2, 1]

# Strategist 
Sources: 

https://github.com/huseinzol05/Stock-Prediction-Models#agents

To Do List:
- [ ] setup basic model
- [ ] get model working with stock of choice
- [ ] connect bot to trading agent
- [ ] get bot making day trades ( can be positive or negative outcome)
- [ ] Optimize that bish


In [254]:
order

Order({   'asset_class': 'us_equity',
    'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
    'canceled_at': None,
    'client_order_id': 'a3c3df84-8cb8-4551-a7cf-c63bb6df6f20',
    'created_at': '2019-03-10T04:22:57.75601Z',
    'expired_at': None,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '35ddc6a5-5333-4ffb-bf1a-3f072a7972b8',
    'limit_price': None,
    'order_type': 'market',
    'qty': '1',
    'side': 'buy',
    'status': 'new',
    'stop_price': None,
    'submitted_at': '2019-03-10T04:22:57.751983Z',
    'symbol': 'TSLA',
    'time_in_force': 'gtc',
    'type': 'market',
    'updated_at': '2019-03-10T04:22:57.761972Z'})

In [10]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz
nyc = timezone('America/New_York')
print(nyc)
today = datetime.now(tz = nyc)
today_str = datetime.now(tz = nyc).strftime('%Y-%m-%d')
calendar = api.get_calendar(start=today_str, end=today_str)[0]
market_open = today.replace(
    hour=calendar.open.hour,
    minute=calendar.open.minute,
    second=0
)
market_open = market_open.astimezone(nyc)
market_close = today.replace(
    hour=calendar.close.hour,
    minute=calendar.close.minute,
    second=0
)
market_close = market_close.astimezone(nyc)

    # Wait until just before we might want to trade
current_dt = datetime.now(tz = nyc)
since_market_open = current_dt - market_open
since_market_open = ts - market_open_dt
until_market_close = market_close_dt - ts
ts = data.start
ts -= timedelta(seconds=ts.second, microseconds=ts.microsecond)
print(since_market_open.seconds )
print(until_market_close.seconds)

America/New_York


NameError: name 'ts' is not defined